In [6]:

from decouple import config
import pandas as pd
import numpy as np
import sys
import glob
import os
sys.path.insert(0, config('PROYECTO_DIR'))
from clases.bd.conexion import  SQLServerConnector
conn = SQLServerConnector()
from sqlalchemy.types import (
    BigInteger, Integer, SmallInteger, String, Date, DateTime, Numeric, CHAR,VARCHAR,DATE,DATETIME,NVARCHAR
)

✅ Conexión exitosa a SQL Server


In [22]:

# Ruta base
ruta_base = config('PROYECTO_DATA') + "/data/CNV/"

# Buscar todos los archivos CSV dentro de la carpeta
archivos_csv = glob.glob(os.path.join(ruta_base, "*.csv"))

# Leer y concatenar todos los archivos
dataframes = []
for archivo in archivos_csv:
    df = pd.read_csv(archivo, sep=",", encoding="utf-8-sig", header=0)  # header=0 usa la primera fila como encabezado
    dataframes.append(df)

    # Unir todos los DataFrames en uno solo
df_cnv = pd.concat(dataframes, ignore_index=True)
#convertir a numeros 
#df['NU_CNV'] = pd.to_numeric(df['NU_CNV'], errors='coerce')
# Mostrar las primeras filas del DataFrame combinado
len(df_cnv)

C:\Users\Irvin\AppData\Local\Temp\ipykernel_9068\1094252495.py:10: DtypeWarning: Columns (15,24,25,26,33,46,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep=",", encoding="utf-8-sig", header=0)  # header=0 usa la primera fila como encabezado
C:\Users\Irvin\AppData\Local\Temp\ipykernel_9068\1094252495.py:10: DtypeWarning: Columns (15,24,25,26,46,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo, sep=",", encoding="utf-8-sig", header=0)  # header=0 usa la primera fila como encabezado


486450

In [23]:
conn.insertar_dataframe(df_cnv, 'TABLA_CNV', if_exists='replace')


✅ DataFrame insertado correctamente en 'TABLA_CNV'
